In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://www.sedaily.com/Stock/Quote?type=1'
html = requests.get(url)
soup = BeautifulSoup(html.text,"html.parser")
soup
        #https://www.sedaily.com/Stock/Quote?type=1에서 html형식으로 텍스트 크롤링


<!DOCTYPE html>

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>전종목 시세 : 서울경제</title>
<meta content="전종목 시세 : 서울경제" name="Author"/>
<meta content="전종목 시세 : 서울경제" name="Keywords"/>
<link href="https://img.sedaily.com/Html/Common/sed.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://img.sedaily.com/Html/Common/sed.ico" rel="icon" type="image/x-icon"/>
<link href="https://img.sedaily.com/Html/Common/favicon-228.png" rel="apple-touch-icon-precomposed" sizes="228x228"/>
<link href="https://img.sedaily.com/Html/Common/favicon-195.png" rel="apple-touch-icon-precomposed" sizes="195x195"/>
<link href="https://img.sedaily.com/Html/Common/favicon-152.png" rel="apple-touch-icon-precomposed" sizes="152x152"/>
<link href="https://img.sedaily.com/Html/Common/favicon-144.png" rel="apple-touch-icon-precomposed" sizes="144x144"/>

In [4]:
all_table = soup.find_all('div',{'class':'table'})
all_table
#크롤링된 텍스트 중 'class'가'table'인 부분 선정

[<div class="table">
 <div>
 <dl class="thead">
 <dt><a href="javascript:window.open('/StockIndex?market=1&amp;tab=2&amp;IndustryCode=018','_blank')" title="">건설</a></dt>
 <dd id="dd_Category_018">
 <span class="down">106.65</span>
 <span class="down"><span>0.46</span></span>
 <span class="down">-0.43%</span>
 </dd>
 </dl>
 <dl class="tbody">
 <dt><a href="javascript:window.open('/Stock/006360','_blank');" title="GS건설">GS건설</a></dt>
 <dd id="dd_Item_006360">
 <span class="down">38,650</span>
 <span class="down"><span>350</span></span>
 <span class="down">-0.90%</span>
 </dd>
 </dl>
 <dl class="tbody">
 <dt><a href="javascript:window.open('/Stock/039570','_blank');" title="HDC아이콘트롤스">HDC아이콘트롤스</a></dt>
 <dd id="dd_Item_039570">
 <span class="up">10,200</span>
 <span class="up"><span>210</span></span>
 <span class="up">+2.10%</span>
 </dd>
 </dl>
 <dl class="tbody">
 <dt><a href="javascript:window.open('/Stock/294870','_blank');" title="HDC현대산업개발">HDC현대산업개발</a></dt>
 <dd id="dd_Item_2948

In [5]:
A = []
for thead in all_table:
    dl = thead.find('dl',{'class':'thead'})
    dt = dl.find('dt')
    fieldName = dt.text
    
    tbody =thead.find_all('dl',{'class':'tbody'})
    # 'thead'하위항목 중 ;class'가 'tbody'인 항목들 찾기
    
    for dl in tbody:
      name=dl.find('dt').get_text()
      dd=dl.find('dd')
      code = dd.get('id').replace('dd_Item_',"")
    # dt의 하위항목 중 id의 "dd_Item_006360"를 공백 처리하고 남은 부분만 list 생성
      A.append(code)
 
      
A

['006360',
 '039570',
 '294870',
 '013580',
 '004200',
 '002990',
 '002995',
 '013700',
 '001260',
 '000210',
 '000215',
 '047040',
 '005960',
 '005965',
 '028100',
 '011160',
 '002410',
 '001470',
 '001880',
 '010960',
 '034300',
 '023960',
 '010400',
 '013360',
 '002780',
 '002787',
 '002785',
 '003070',
 '003075',
 '009410',
 '009415',
 '014790',
 '004960',
 '051600',
 '097230',
 '000720',
 '000725',
 '002460',
 '006840',
 '027410',
 '138930',
 '001040',
 '001045',
 '000590',
 '016610',
 '005830',
 '139130',
 '004840',
 '078930',
 '078935',
 '012630',
 '175330',
 '096760',
 '105560',
 '009440',
 '030210',
 '003550',
 '003555',
 '006260',
 '229640',
 '005940',
 '005945',
 '036530',
 '034730',
 '006120',
 '006125',
 '03473K',
 '001510',
 '001515',
 '030610',
 '024110',
 '000320',
 '000325',
 '072710',
 '084690',
 '084695',
 '003540',
 '003547',
 '003545',
 '003090',
 '000640',
 '082640',
 '013570',
 '000400',
 '004990',
 '00499K',
 '088980',
 '138040',
 '008560',
 '000060',
 '006800',

In [6]:
A.remove('316140')
A.remove('307950')
A.remove('298690')
A.remove('00806K')
A.remove('192650')
A.remove('069460')
#일부 불완전 주가 데이터 제외 

In [ ]:

import requests
import pandas as pd
import xml.etree.ElementTree as ET


def getStockPrice(StockCode, Count = 1825+20) :
    ret = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close','volumn'])
    
    url = "https://fchart.stock.naver.com/sise.nhn?symbol={}&timeframe=day&count={}&requestType=0"
    url = url.format(StockCode, Count)
    response = requests.get(url)
    
    e = ET.fromstring(response.text)
    
    stock_history = []
    for item in e.find('chartdata').findall('item') :
        data = item.get('data').split('|')
        tmp = [data[0],int(data[1]),int(data[2]),int(data[3]),int(data[4]),int(data[5])]
        
        if data[0] >= '20140501' and data[0] <= '20190515' :
          ret.loc[len(ret)] = tmp

    return ret

In [13]:
import requests
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from urllib.request import urlopen
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [ ]:
a= []
for i in range(len(A)):
    
    stock = getStockPrice(A[i])
    stock = stock.replace(to_replace = 'None', value = np.nan).dropna()
    
    a.append(stock)
    
A = pd.DataFrame(a)
A.to_csv('stock.csv', index = False)